In [1]:
#引用库
import sys
import numpy as np
import os 
import pandas as pd
from scipy.signal import find_peaks_cwt
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#将text数据转化为numpy数组
def text_data_to_numpy_array(filepath):
    """
    Transforms the data saved as a text or csv file to a numpy array.

    Parameters
    ----------
    filepath : string
        The path of the text or csv file
    filetype : string
        The type of the files (tex or csv)

    Returns
    -------
    data : numpy array
        Numpy array with two rows, given by the x and y values of the data
    """

    filetype = filepath[-3:]
    if filetype == "csv":
        delim = ','
    elif filetype == "txt":
        delim = '\t'

    f = open(filepath, 'r')
    X, Y = [], []
    for row in f:
        xi, yi = row.split(delim)
        X.append(float(xi))
        Y.append(float(yi))
    data = np.array([X, Y])
    return data

#
def extract_data(folder):
    """
    Extracts the data from all the files in the folder, transforms them into
    arrays, and puts them in a dictionary.

    Parameters
    ----------
    folder : string
        The path of the folder

    Returns
    -------
    data_in_folder : dictionary
        The keys of data_in_folder are the names of the files in the folder,
        the values are the corresponding data, saved as a numpy array with two
        rows, the first containing the x axis, athe second the y axis
    """

    data_in_folder = {}

    for file_name in os.listdir(folder):

        try:
#            print(folder + '\\' + file_name)
            data_in_folder[file_name[:-4]] = \
                text_data_to_numpy_array(folder + '/'+ file_name)
                
        except:
            print("Problem with file", file_name)

    return data_in_folder


In [3]:
#预处理，提取数据
def same_scale(data_train):
    min=3000
    max=0
    for name in data_train:
        if min>data_train[name][0,0]:
            min=int(data_train[name][0,0])
        if max<data_train[name][0,-1]:
            max=int(data_train[name][0,-1])
    #padding
    #为了不丢失数据，拼接为最大矩阵
    for name in data_train:
        head=int(data_train[name][0,0])
        if head>min:
            k1=np.array([i for i in range(min,head)]).reshape((1,head-min))
            k2=np.zeros((1,head-min))
            k=np.vstack((k1,k2))
            data_train[name]=np.hstack((k,data_train[name]))
            
        tail=int(data_train[name][0,-1])
        if tail<max:
            c1=np.array([i for i in range(tail+1,max+1)]).reshape((1,max-tail))
            c2=np.zeros((1,max-tail))
            c=np.vstack((c1,c2))
            data_train[name]=np.hstack((data_train[name],c))
            
    return data_train       
#这个是用线性插值得到序列的拼接
def preprocess(data, lb, up, res = 1):
    '''
    truncate, smooth and normalize the signal in place.
    '''
    x_domain = np.arange(lb, up, res)
    for key, value in data.items():
            mask = (value[0,:]>lb) & (value[0,:] < up)
    #        data[key] = np.stack((value[0, mask], smooth1D(value[1,mask]/np.linalg.norm(value[1, mask]), window_len)[half_window_len:-half_window_len]))
#            data[key] = np.stack((value[0, mask], value[1,mask]/np.linalg.norm(value[1, mask])))
            data[key] = np.stack((value[0, mask], value[1,mask]/np.max(value[1, mask])))
    #        data[key] = square_rescale(value[:,mask])
            f = interp1d(value[0,mask], data[key], 'linear', fill_value= 'extrapolate')
            data[key] = f(x_domain)
            
    return data
#去除基线
from BaselineRemoval import BaselineRemoval
#gituhub 的一个包，有三个去基线方法(readme)
#usage：
# input_array=[10,20,1.5,5,2,9,99,25,47]
# polynomial_degree=2 #only needed for Modpoly and IModPoly algorithm

# baseObj=BaselineRemoval(input_array)
# Modpoly_output=baseObj.ModPoly(polynomial_degree)
# Imodpoly_output=baseObj.IModPoly(polynomial_degree)
#Zhangfit_output=baseObj.ZhangFit()



In [4]:
#DP algorithm
def Dot(x, y):
    return np.sum(x*y)/np.linalg.norm(x)/np.linalg.norm(y)
#correlation
def CC(x, y):
    '''
    cross-corelation, x, y are two vectors
    '''
    return np.corrcoef(x, y)[0,1]
#欧拉距离
def ED(x,y):
    return np.linalg.norm(np.array(x)-np.array(y))
#def dist

In [5]:
#提取数据
data_train=extract_data('data_train')     #这是个字典
data_train=preprocess(data_train,200,3500)

data_test=extract_data('data_test')
data_test=preprocess(data_test,200,3500)

In [6]:
#全体去除基线的数据集
from BaselineRemoval import BaselineRemoval
for name in data_test:
    data_train[name]=data_test[name]
    
data_br_train={}
degree=2
for name in data_train:
    base=BaselineRemoval(data_train[name][1])
    data_br_train[name]=base.ModPoly(degree)
data_br_test={}
for name in data_test:
    base=BaselineRemoval(data_test[name][1])
    data_br_test[name]=base.ModPoly(degree)
#去除后的数据集是每个name对应一个y序列的，不是之前的data_train数据集那样是一个name对应一个x序列和y序列
#这里去除基线只用了一种方法modpoly，因为能尽可能的保存波图的的信息

        

In [7]:
#DTW
import dtw as dtw
#以2500为分成两段，两段式，对曲线f，和某个阈值y和L, 找到集合X = {f>y}， 就选取[min(X)-L, max(X)+L]
def find_area(data,X,thredhold,cut,L):
    area={}
    for name in data:
        temp1=[]#临时
        temp2=[]
        #存下index
        for i in range(0,cut-X[0]):
            if data[name][i]>=thredhold:
                temp1.append(i)
                #得到y>thredhold对应的列表index
        
        #min1      
        if temp1!=[]and (X[temp1[0]]-L)<=X[0]:
            temp2.append(X[0])
        if  temp1!=[]and (X[temp1[0]]-L)>X[0]:
            temp2.append(X[temp1[0]]-L)
        #max1
        if  temp1!=[]and(X[temp1[-1]]+L)>=cut:
            temp2.append(cut-1)
        if temp1!=[]and (X[temp1[-1]]+L)<cut:
            temp2.append(X[temp1[-1]]+L)
            
        temp1=[]
        for k in range(cut-X[0],len(data[name])):
            if data[name][k]>=thredhold:
                temp1.append(k)
        #min2
        if temp1!=[] and (X[temp1[0]]-L)<=cut:
            temp2.append(cut)
        if temp1!=[] and (X[temp1[0]]-L)>cut:
            temp2.append(X[temp1[0]]-L)
        #max2
        if temp1!=[] and (X[temp1[-1]]+L)>=X[-1]:
            temp2.append(X[-1])
        if temp1!=[] and (X[temp1[-1]]+L)<X[-1]:
            temp2.append(X[temp1[-1]]+L)     
        area[name]=temp2
    #找area——x对应的y
    y={}
    for name in area:
        temp=[]
        for i in range (0,len(area[name]),2):
            k=area[name][i]-X[0]
            j=area[name][i+1]-X[0]
            temp.append(data[name][k:j])
            
        y[name]=temp
    
    return area,y


In [ ]:
#dtw procedure
'''def dtw(x, y):
    # Initialization
    for i = 1..n
        for j = 1..m
            C[i, j] = inf

    C[0, 0] = 0.

   # Main loop
   for i = 1..n
        for j = 1..m
            dist = d(x_i, y_j) ** 2
            C[i, j] = dist + min(C[i-1, j], C[i, j-1], C[i-1, j-1])

    return sqrt(C[n, m])'''

from dtw import dtw
def match_dtw(data_dtw,data_test,X):
#这里的data只用去基线后的
    manhatan= lambda x, y: np.abs(x - y)
    result={}
    for name in data_test:
        temp=[]
        for name2 in data_dtw:

            if len(data_dtw[name])!=len(data_dtw[name2]):
                temp.append(-1)
                continue
            temp2=[]
            for i in range(0,len(data_dtw[name])):
                #print(len(np.float32(data_dtw[name2][i])),name,name2,i)
                temp2.append(dtw(np.float32(data_dtw[name][i]),np.float32(data_dtw[name2][i]),dist=ED)[0])#这里改float32试试
            temp.append(sum(temp2))
        result[name]=temp
    return result    
            
    
name_list=[]  
for name in data_test:
    name_list.append(name)

#这里的data只用去基线后的
X=[i for i in range(200,3501)]
area,data_dtw=find_area(data_br_train,X,0.18,2500,100)
result=match_dtw(data_dtw,name_list[12:-1],X)


In [ ]:
#保存数据，这个数据跑的好久啊
def save(result):
    with open('data_dtw_3.txt','w') as f:
        for name in result:
            f.write(name+'\t')
            if np.shape(result[name])==():
                f.write(str(result[name]))
            else:
                for i in result[name]:
                    f.write(str(i)+'\t')
            f.write('\n')
save(result)


In [ ]:
test={}
for name in result:
    test[name]=result[name]

In [ ]:
def one_hot(dist,thredhold):
    temp=[]
    for i in dist:
        if i<thredhold and i>=0:
            temp.append(1)
        else:
            temp.append(-1)
    return temp

def accuracy_one(onehot):
    tp=1
    tn=0
    for i in onehot:
        if i==-1:
            tn=tn+1

    return (tn+tp)/len(onehot)

In [ ]:
final={}
for name in result:
    final[name]=accuracy_one(one_hot(result[name],13))
print (final)

In [ ]:
#找到符合阈值内的几个名字，也就是结果
def find_nn(result,data_dtw,thredhold,name):
    k=0
    K=[]
    for res in result[name]:
        if res<=thredhold and res>=0:
            K.append(k)
        k=k+1
        
    name_list=[]
    for k in K:
        i=0
        for name in data_dtw:
            if i==k:
                name_list.append(name)
            i=i+1
    return name_list


In [1]:
#误差分析啊
import re
with open ('data_dtw_3.txt','r')as f:
    t=f.read()
    t=t.split('\t')
#没有取消\n，多个用re正则表达式

In [2]:
k=0
test={}
load_data=[]
for i in t:
    if k%207==0:
        name=i
    else:
        load_data.append(float(i))
    if k%207==206:
        test[name]=load_data
        load_data=[]
    k=k+1
#去除\n
i=0
test1={}
for name in test:
    if i==0:
        test1[name]=test[name]
        i=i+1
        continue
    k=re.split('\n',name)
    test1[k[1]]=test[name]
    i=i+1

In [3]:
final={}
for name in test1:
    final[name]=accuracy_one(one_hot(test1[name],30))
print (final)

NameError: name 'accuracy_one' is not defined

In [ ]:
#boxplot 箱形图
temp1=[]
for name in final:
    temp1.append(final[name])
import seaborn as sns
fig=plt.figure(figsize=(10,10))
sns.boxplot(temp1,orient='v',width=0.5,color='g')
plt.xlabel('',fontsize=30)
plt.ylabel('accuracy',fontsize=10)
plt.savefig('DTE_acc.png') 
